In [ ]:

from glob import glob
import json
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf


repo_path = ".."
import sys
sys.path.append(f"{repo_path}/code/")

font = {'size'   : 14}

matplotlib.rc('font', **font)
figure = {'figsize'   : (12,8),
          'max_open_warning': False}
matplotlib.rc('figure', **figure)

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))

inf_history_path = ("../data/models/b_2/"
                    "ne_200_lr_1e-06_bs_1000_t_0.1/"
                    "init_[0-9]/history.json")


for h_path in glob(inf_history_path):
  with open(h_path) as h_file:
    h_dict  = json.load(h_file)
  
  valid_step, valid_loss = np.array(h_dict["loss_valid"]).T

  ax.plot(valid_loss)
  
ax.set_xlabel("training epoch")
ax.set_ylabel("validation-set inference-aware loss")

fig.savefig("../paper/gfx/figure4a.pdf",bbox_inches='tight')
fig

In [ ]:
from template_model import TemplateModel
from glob import glob

from importlib import reload
import template_model
reload(template_model)

tm = TemplateModel(multiple_pars=True)

inf_template_path = "../data/models/b_2/ne_200_lr_1e-06_bs_1000_t_0.1/init_[0-9]/templates.json"
clf_template_path = "../data/models/cross_entropy/ne_200_lr_0.001_bs_32/init_[0-9]/templates.json"

In [ ]:

step_size = 0.1
n_steps = 100
d = 0.0
pars = ["r_dist","b_rate"]

s_exp_scan = np.linspace(20.,80.,61, endpoint=True)
par_phs = {tm.r_dist : 2.0*np.ones_like(s_exp_scan),
           tm.b_rate : 3.0*np.ones_like(s_exp_scan),
           tm.s_exp :  s_exp_scan,
           tm.b_exp : 1000.0*np.ones_like(s_exp_scan)} 


inf_pls = {}
for inf_path in glob(inf_template_path):
  with tf.Session() as sess:
    tm.templates_from_json(inf_path)
    # get asimov data (default pars)
    asimov_data = tm.asimov_data(sess=sess)

    obs_phs = {tm.obs : asimov_data}
    mod_phs = par_phs.copy()
    # get likelihood before changing pars
    nll, sub_hess, sub_grad = tm.hessian_and_gradient(pars=pars,
                                                      par_phs=mod_phs, obs_phs=obs_phs)
    # profile likelihood with Newton method
    for i in range(n_steps):
      newton_step =  np.matmul(np.linalg.inv(sub_hess+d*np.ones([len(pars)])),sub_grad[:,:,np.newaxis])
      mod_phs[tm.r_dist] = mod_phs[tm.r_dist] - step_size*newton_step[:,0,0]
      mod_phs[tm.b_rate] = mod_phs[tm.b_rate] - step_size*newton_step[:,1,0]
      p_nll, sub_hess, sub_grad = tm.hessian_and_gradient(pars=["r_dist","b_rate"],
                                                        par_phs=mod_phs, obs_phs=obs_phs)
    
    inf_pls[inf_path] = p_nll
    print("nll - p_nll",(p_nll-nll).sum())

In [ ]:
step_size = 0.1
n_steps = 100
d = 0.0
pars = ["r_dist","b_rate"]


s_exp_scan = np.linspace(20.,80.,61, endpoint=True)
par_phs = {tm.r_dist : 2.0*np.ones_like(s_exp_scan),
           tm.b_rate : 3.0*np.ones_like(s_exp_scan),
           tm.s_exp :  s_exp_scan,
           tm.b_exp : 1000.0*np.ones_like(s_exp_scan)} 


clf_pls = {}
for clf_path in glob(clf_template_path):
  with tf.Session() as sess:
    templates = tm.templates_from_json(clf_path)
    # get asimov data (default pars)
    asimov_data = tm.asimov_data(sess=sess)
    obs_phs = {tm.obs : asimov_data}
    mod_phs = par_phs.copy()
    # get likelihood before changing pars
    nll, sub_hess, sub_grad = tm.hessian_and_gradient(pars=["r_dist","b_rate"],
                                                      par_phs=mod_phs, obs_phs=obs_phs)
    # profile likelihood with Newton method
    for i in range(n_steps):
      newton_step =  np.matmul(np.linalg.inv(sub_hess+d*np.eye(len(pars))),sub_grad[:,:,np.newaxis])
      mod_phs[tm.r_dist] = mod_phs[tm.r_dist] - step_size*newton_step[:,0,0]
      mod_phs[tm.b_rate] = mod_phs[tm.b_rate] - step_size*newton_step[:,1,0]
      p_nll, sub_hess, sub_grad = tm.hessian_and_gradient(pars=["r_dist","b_rate"],
                                                        par_phs=mod_phs, obs_phs=obs_phs)

    
    clf_pls[clf_path] = p_nll
    print("nll - p_nll",(p_nll-nll).sum())

In [ ]:
from scipy.interpolate import InterpolatedUnivariateSpline
fig, ax = plt.subplots(figsize=(8,6))

inf_roots = {}
for name, p_nll in inf_pls.items():
  shift_nll = p_nll-p_nll.min()
  inf_roots[name] = InterpolatedUnivariateSpline(s_exp_scan, shift_nll-0.5).roots()
  inf_line = ax.plot(s_exp_scan, shift_nll ,"b",alpha=0.3)

clf_roots = {}
for name, p_nll in clf_pls.items():
  shift_nll = p_nll-p_nll.min()
  clf_roots[name] = InterpolatedUnivariateSpline(s_exp_scan, shift_nll-0.5).roots()
  clf_line = ax.plot(s_exp_scan, shift_nll,"r", alpha=0.3)

ax.set_ylim(bottom=0)
ax.set_xlim(left=s_exp_scan.min(), right=s_exp_scan.max())

ax.set_xlabel("$s$ parameter of interest")
ax.set_ylabel(r"profiled likelihood $\Delta(\mathcal{-\ln L})$")


inf_mean = np.mean([(r[1]-r[0])/2. for r in inf_roots.values()])
inf_std = np.std([(r[1]-r[0])/2. for r in inf_roots.values()],ddof=1)
clf_mean = np.mean([(r[1]-r[0])/2. for r in clf_roots.values()])
clf_std = np.std([(r[1]-r[0])/2. for r in clf_roots.values()],ddof=1)
print(f"INFERNO widths {inf_mean} pm {inf_std}")
print(f"NN Classifier widths {clf_mean} pm {clf_std}")


ax.legend((clf_line[0], inf_line[0]), ("cross-entropy","inference-aware"),
          loc="upper center",frameon=False)


fig.savefig("../paper/gfx/figure4b.pdf",bbox_inches='tight')


fig